In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Performance metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

from preprocessing import create_and_preprocess_X_y, create_train_test_set
from model import create_y_pred_baseline


# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import LSTM, Flatten, Dense

2023-09-13 10:32:16.750339: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 10:32:16.751718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 10:32:16.788136: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 10:32:16.789081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 10:32:17.451885: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Load and preprocess the data using create_and_preprocess_X_y()
X, y = create_and_preprocess_X_y()

In [3]:
X.shape, y.shape

((88, 7400, 106), (88, 100, 106))

In [4]:
# Split the data into train and test sets using create_train_test_set()
X_train, X_test, y_train, y_test = create_train_test_set(X, y, train_size=0.8)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((70, 7400, 106), (18, 7400, 106), (70, 100, 106), (18, 100, 106))

In [9]:
# Define the LSTM model
model = Sequential([
    layers.LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.LSTM(units=64, return_sequences=True),
    layers.TimeDistributed(layers.Dense(units=106))
])

In [10]:
# Compile the model
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mse'])



In [11]:
# Train the model

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1) 

Epoch 1/10


ValueError: in user code:

    File "/home/lydia/.pyenv/versions/3.10.6/envs/ChopAI/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/lydia/.pyenv/versions/3.10.6/envs/ChopAI/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/lydia/.pyenv/versions/3.10.6/envs/ChopAI/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/lydia/.pyenv/versions/3.10.6/envs/ChopAI/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/lydia/.pyenv/versions/3.10.6/envs/ChopAI/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compi

In [12]:
# Make predictions test 
y_test_pred = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [13]:
# Calculate performance metrics
mse = mean_squared_error(y_test.reshape(-1, y_test.shape[-1]), y_test_pred.reshape(-1, y_test_pred.shape[-1]))
r2 = r2_score(y_test.reshape(-1, y_test.shape[-1]), y_test_pred.reshape(-1, y_test_pred.shape[-1]))

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2) Score: {r2}")

ValueError: Found input variables with inconsistent numbers of samples: [1800, 133200]